In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as pyplot
import os

In [2]:
X = np.load('/content/drive/MyDrive/Image Processing and Computer Vision/Project/optical_flows.npy')  # Load optical flows data
y = np.load('/content/drive/MyDrive/Image Processing and Computer Vision/Project/labels.npy')  # Load labels dat

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, GlobalAveragePooling2D # type: ignore

# Define a custom input layer for 2 channels
input_tensor = Input(shape=(224, 224, 2))

# Load the VGG16 model without the top layers (use the functional API)
# We use input_tensor instead of the original input layer of VGG16
x = Conv2D(64, (3,3), activation='relu', padding='same', name='block_conv1')(input_tensor)
base_model = VGG16(weights='imagenet', include_top=False)

for layer in base_model.layers[2:]:
    x = layer(x)

# Add custom layers on top of the base model
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)  # 4 classes for emotions: 0, 1, 2, 3

# Create the new model
model = Model(inputs=input_tensor, outputs=output_layer)

# Freeze the layers in the base model
for layer in base_model.layers[1:]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary to verify the architecture
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 2)]     0         
                                                                 
 block_conv1 (Conv2D)        (None, 224, 224, 64)      1216      
                                                                 
 block1_conv2 (Conv2D)       multiple                  36928     
                                                                 
 block1_pool (MaxPooling2D)  multiple                  0         
                                                                 
 block2_conv1 (Conv2D)       multiple                  73856     
                                                                 
 block2_conv2 (Conv2D)       multiple                  147584    
                                                              

In [4]:
#Shuffle data
from sklearn.utils import shuffle # type: ignore

# Shuffle X and y
X, y = shuffle(X, y, random_state=42)

In [7]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Increase to 20 epochs
    batch_size=16,
    verbose=1,
)


model.save('emotion_classification_model.h5')

Epoch 1/20
24/24 [==============================] - 47s 2s/step - loss: 1.2090 - accuracy: 0.4479 - val_loss: 0.9190 - val_accuracy: 0.5938
Epoch 2/20
24/24 [==============================] - 45s 2s/step - loss: 0.8319 - accuracy: 0.6693 - val_loss: 0.7293 - val_accuracy: 0.7396
Epoch 3/20
24/24 [==============================] - 45s 2s/step - loss: 0.6822 - accuracy: 0.7734 - val_loss: 0.6245 - val_accuracy: 0.6979
Epoch 4/20
24/24 [==============================] - 45s 2s/step - loss: 0.5480 - accuracy: 0.8333 - val_loss: 0.5135 - val_accuracy: 0.7708
Epoch 5/20
24/24 [==============================] - 45s 2s/step - loss: 0.4433 - accuracy: 0.8490 - val_loss: 0.4289 - val_accuracy: 0.8021
Epoch 6/20
24/24 [==============================] - 45s 2s/step - loss: 0.3591 - accuracy: 0.8880 - val_loss: 0.3694 - val_accuracy: 0.8750
Epoch 7/20
24/24 [==============================] - 44s 2s/step - loss: 0.3020 - accuracy: 0.9062 - val_loss: 0.3564 - val_accuracy: 0.8542
Epoch 8/20
24/24 [==

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
